# Feature Engineering

## 0. Carga inicial de los datasets

Este notebook comienza con la carga explícita de los datasets previamente utilizados en el análisis exploratorio.

No se asume ninguna variable definida previamente.  
Todas las estructuras necesarias se inicializan en este punto.


In [3]:
import pandas as pd
from pathlib import Path


### 0.1 Definición de la ruta de los datos

Se define la ruta base donde se encuentran los archivos de datos utilizados en el proyecto.


In [4]:
DATA_PATH = Path("../data")


### 0.2 Carga de los datasets

Se cargan explícitamente todos los datasets que formarán parte del proceso de Feature Engineering.
Cada dataset se almacena en un diccionario para facilitar su manipulación posterior.


In [8]:
path_psnw = r"C:\Users\rosal\Desktop\practica-1-EDA-pj2r98\data\PSNW.csv"
path_labor_slack = r"C:\Users\rosal\Desktop\practica-1-EDA-pj2r98\data\Labor Slack.csv"
path_inflation = r"C:\Users\rosal\Desktop\practica-1-EDA-pj2r98\data\Inflation Monthly Euro Zone.csv"
path_gdp_growth = r"C:\Users\rosal\Desktop\practica-1-EDA-pj2r98\data\GDP Growth Rate.csv"
path_psnw_gdp = r"C:\Users\rosal\Desktop\practica-1-EDA-pj2r98\data\EU PSNW over GDP.csv"
path_debt_gdp = r"C:\Users\rosal\Desktop\practica-1-EDA-pj2r98\data\Debt-To-GDP Ratio.csv"

In [9]:
#Carga del Dataset 

df_psnw = pd.read_csv(path_psnw)
df_labor_slack = pd.read_csv(path_labor_slack)
df_inflation = pd.read_csv(path_inflation)
df_gdp_growth = pd.read_csv(path_gdp_growth)
df_psnw_gdp = pd.read_csv(path_psnw_gdp)
df_debt_gdp = pd.read_csv(path_debt_gdp)


### 0.3 Construcción del diccionario de datasets

Se agrupan los datasets cargados individualmente en una estructura común (`dataframes`)
para facilitar las operaciones posteriores de Feature Engineering.


In [15]:
dataframes = {
    "PSNW": df_psnw,
    "Labor_Slack": df_labor_slack,
    "Inflation_Monthly_Euro_Zone": df_inflation,
    "GDP_Growth_Rate": df_gdp_growth,
    "EU_PSNW_over_GDP": df_psnw_gdp,
    "Debt_to_GDP_Ratio": df_debt_gdp
}


### 0.4 Verificación de la carga de los datasets

Se verifica el número de observaciones y columnas de cada dataset cargado.


In [16]:
{key: df.shape for key, df in dataframes.items()}


{'PSNW': (3720, 15),
 'Labor_Slack': (11856, 13),
 'Inflation_Monthly_Euro_Zone': (20135, 10),
 'GDP_Growth_Rate': (958, 10),
 'EU_PSNW_over_GDP': (1740, 29),
 'Debt_to_GDP_Ratio': (1260, 11)}

### 0.5. Identificación de columnas comunes entre datasets

Se identifican las columnas compartidas entre todos los datasets para establecer
la base estructural mínima necesaria para una posterior integración.


In [17]:
columnas_por_dataset = {
    key: set(df.columns)
    for key, df in dataframes.items()
}

columnas_comunes = set.intersection(*columnas_por_dataset.values())
columnas_comunes


{'CONF_STATUS', 'OBS_FLAG', 'OBS_VALUE', 'TIME_PERIOD', 'freq', 'geo', 'unit'}

### 0.6. Clasificación de columnas administrativas y analíticas

Se distinguen columnas de tipo administrativo o metadata de aquellas con
contenido analítico relevante para el modelado.


In [19]:
columnas_administrativas = {
    "DATAFLOW",
    "LAST UPDATE",
    "freq",
    "na_item",
    "sector",
    "unit",
    "CONF_STATUS",
    "OBS_FLAG",
    "Confidentiality status (flag)",
    "Observation value"
}

columnas_analiticas_por_dataset = {
    key: [col for col in df.columns if col not in columnas_administrativas]
    for key, df in dataframes.items()
}

columnas_analiticas_por_dataset


{'PSNW': ['finpos',
  'stk_flow',
  'sector2',
  'co_nco',
  'geo',
  'TIME_PERIOD',
  'OBS_VALUE'],
 'Labor_Slack': ['s_adj',
  'wstatus',
  'sex',
  'age',
  'geo',
  'TIME_PERIOD',
  'OBS_VALUE'],
 'Inflation_Monthly_Euro_Zone': ['coicop', 'geo', 'TIME_PERIOD', 'OBS_VALUE'],
 'GDP_Growth_Rate': ['geo', 'TIME_PERIOD', 'OBS_VALUE'],
 'EU_PSNW_over_GDP': ['STRUCTURE',
  'STRUCTURE_ID',
  'STRUCTURE_NAME',
  'Time frequency',
  'National accounts indicator (ESA 2010)',
  'finpos',
  'Financial position',
  'stk_flow',
  'Stock or flow',
  'Sector',
  'sector2',
  'Counterpart sector',
  'co_nco',
  'Consolidated/Non consolidated',
  'Unit of measure',
  'geo',
  'Geopolitical entity (reporting)',
  'TIME_PERIOD',
  'Time',
  'OBS_VALUE',
  'Observation status (Flag) V2 structure'],
 'Debt_to_GDP_Ratio': ['geo', 'TIME_PERIOD', 'OBS_VALUE']}

### 0.7. Definición de columnas base para integración

Se definen las columnas mínimas necesarias para integrar los datasets en una
estructura común.


In [20]:
columnas_base = ["geo", "TIME_PERIOD", "OBS_VALUE"]

columnas_base


['geo', 'TIME_PERIOD', 'OBS_VALUE']

### 0.8. Reducción estructural de los datasets

Se conservan únicamente las columnas necesarias para la integración y el
posterior modelado.


In [21]:
dataframes_reducidos = {}

for key, df in dataframes.items():
    columnas_existentes = [col for col in columnas_base if col in df.columns]
    dataframes_reducidos[key] = df[columnas_existentes].copy()

{key: df.shape for key, df in dataframes_reducidos.items()}


{'PSNW': (3720, 3),
 'Labor_Slack': (11856, 3),
 'Inflation_Monthly_Euro_Zone': (20135, 3),
 'GDP_Growth_Rate': (958, 3),
 'EU_PSNW_over_GDP': (1740, 3),
 'Debt_to_GDP_Ratio': (1260, 3)}

### 0.9. Renombrado de variables observadas

Se renombra la columna `OBS_VALUE` para reflejar el contenido económico de cada dataset.


In [22]:
nombres_variables = {
    "PSNW": "psnw",
    "Labor_Slack": "labor_slack",
    "Inflation_Monthly_Euro_Zone": "inflation",
    "GDP_Growth_Rate": "gdp_growth",
    "EU_PSNW_over_GDP": "psnw_gdp_ratio",
    "Debt_to_GDP_Ratio": "debt_gdp_ratio"
}

for key, df in dataframes_reducidos.items():
    if "OBS_VALUE" in df.columns:
        df.rename(columns={"OBS_VALUE": nombres_variables[key]}, inplace=True)

dataframes_reducidos


{'PSNW':            geo TIME_PERIOD      psnw
 0      Austria     2018-Q1 -197341.8
 1      Austria     2018-Q2 -198387.8
 2      Austria     2018-Q3 -190542.5
 3      Austria     2018-Q4 -194800.6
 4      Austria     2019-Q1 -197411.8
 ...        ...         ...       ...
 3715  Slovakia     2024-Q2     -38.9
 3716  Slovakia     2024-Q3     -39.5
 3717  Slovakia     2024-Q4     -42.0
 3718  Slovakia     2025-Q1     -41.9
 3719  Slovakia     2025-Q2     -42.6
 
 [3720 rows x 3 columns],
 'Labor_Slack':            geo TIME_PERIOD  labor_slack
 0      Austria     2009-Q1          2.3
 1      Austria     2009-Q2          2.0
 2      Austria     2009-Q3          2.2
 3      Austria     2009-Q4          2.3
 4      Austria     2010-Q1          2.2
 ...        ...         ...          ...
 11851  Türkiye     2024-Q2          7.7
 11852  Türkiye     2024-Q3          7.7
 11853  Türkiye     2024-Q4          7.7
 11854  Türkiye     2025-Q1          7.2
 11855  Türkiye     2025-Q2          7.5
 

### 0.10. Estado de los datasets antes de la integración

Los datasets se encuentran estructuralmente alineados y listos para ser integrados
en un único dataset maestro.


In [23]:
{key: df.head() for key, df in dataframes_reducidos.items()}


{'PSNW':        geo TIME_PERIOD      psnw
 0  Austria     2018-Q1 -197341.8
 1  Austria     2018-Q2 -198387.8
 2  Austria     2018-Q3 -190542.5
 3  Austria     2018-Q4 -194800.6
 4  Austria     2019-Q1 -197411.8,
 'Labor_Slack':        geo TIME_PERIOD  labor_slack
 0  Austria     2009-Q1          2.3
 1  Austria     2009-Q2          2.0
 2  Austria     2009-Q3          2.2
 3  Austria     2009-Q4          2.3
 4  Austria     2010-Q1          2.2,
 'Inflation_Monthly_Euro_Zone':        geo TIME_PERIOD  inflation
 0  Albania     2018-01        1.7
 1  Albania     2018-02        1.0
 2  Albania     2018-03        0.3
 3  Albania     2018-04       -0.4
 4  Albania     2018-05       -1.4,
 'GDP_Growth_Rate':   geo  TIME_PERIOD  gdp_growth
 0  AL         2013         1.7
 1  AL         2014         2.2
 2  AL         2015         2.2
 3  AL         2016         3.9
 4  AL         2017         3.3,
 'EU_PSNW_over_GDP':   geo TIME_PERIOD  psnw_gdp_ratio
 0  AT     2018-Q1           -53.1
 1  A

## 1. Introducción al Feature Engineering


En este notebook se preparan las variables que serán utilizadas por los modelos de Machine Learning supervisado.

El objetivo de esta etapa no es analizar ni interpretar los datos, sino dejarlos en una forma estructurada, consistente y utilizable para el entrenamiento de modelos, partiendo del dataset maestro construido a partir de las fuentes originales.


### 1.1 Objetivo del Feature Engineering


El objetivo de esta fase es transformar el dataset maestro en un conjunto de variables explicativas adecuado para el modelado, asegurando:

- coherencia estructural,
- compatibilidad entre variables,
- ausencia de ruido innecesario,
- y preparación para los algoritmos de aprendizaje supervisado.


### 1.2 Contexto dentro del flujo del proyecto


Este notebook se apoya directamente en los resultados del EDA y precede a la fase de modelado.

Todas las transformaciones realizadas aquí afectan de forma directa el desempeño, estabilidad e interpretabilidad de los modelos, por lo que cada paso se documenta y valida antes de continuar.


## 2. Construcción del dataset maestro unificado

A partir de los datasets reducidos y estandarizados, se procede a la construcción de un dataset maestro unificado.

El objetivo de esta etapa es integrar todas las variables explicativas en una única estructura común que sirva como base para el proceso de Feature Engineering y el posterior modelado supervisado.

La integración se realiza manteniendo una fila por combinación (geo, TIME_PERIOD) y una columna por variable económica.


## 2.1 Unidad de observación

Cada observación del dataset maestro representa una entidad geográfica (geo) asociada a un periodo discreto (TIME_PERIOD).

Esta definición de unidad de observación se mantiene constante durante todo el proceso de Feature Engineering y modelado, garantizando coherencia estructural y compatibilidad entre variables.


## 2.2 Integración progresiva de datasets en un dataset maestro

A partir de los datasets reducidos y renombrados, se procede a la integración progresiva en un único dataset maestro.

La integración se realiza utilizando las claves comunes (geo, TIME_PERIOD), garantizando una estructura coherente y alineada entre todas las variables explicativas.

En esta etapa no se aplican transformaciones estadísticas ni imputaciones.


## 2.2.1 Diagnóstico de duplicados por claves estructurales

Antes de la integración, se verifica si cada dataset contiene una única observación por combinación (geo, TIME_PERIOD).


In [28]:
{
    key: df.duplicated(subset=["geo", "TIME_PERIOD"]).sum()
    for key, df in dataframes_reducidos.items()
}


{'PSNW': np.int64(2790),
 'Labor_Slack': np.int64(9444),
 'Inflation_Monthly_Euro_Zone': np.int64(16022),
 'GDP_Growth_Rate': np.int64(460),
 'EU_PSNW_over_GDP': np.int64(870),
 'Debt_to_GDP_Ratio': np.int64(0)}

## 2.2.2 Resolución estructural de duplicados

Cuando un dataset presenta múltiples observaciones para una misma combinación (geo, TIME_PERIOD), se colapsan mediante agregación promedio, preservando una única observación por unidad.


In [29]:
dataframes_sin_duplicados = {}

for key, df in dataframes_reducidos.items():
    dataframes_sin_duplicados[key] = (
        df
        .groupby(["geo", "TIME_PERIOD"], as_index=False)
        .mean(numeric_only=True)
    )


### 2.3 Normalización estructural de la variable TIME_PERIOD

Dado que los datasets presentan diferentes representaciones temporales
(anual, trimestral, mensual), se normaliza `TIME_PERIOD` a un formato de texto homogéneo.

Esta normalización no implica tratamiento temporal ni modelado de series de tiempo.
La variable representa únicamente una **clave de periodo económico**.


In [32]:
def normalizar_time_period(df):
    df = df.copy()
    df["TIME_PERIOD"] = df["TIME_PERIOD"].astype(str)
    return df

dataframes_sin_duplicados = {
    key: normalizar_time_period(df)
    for key, df in dataframes_sin_duplicados.items()
}


### 2.4 Construcción del dataset maestro unificado

Una vez garantizada la unicidad de las claves y la coherencia estructural de las variables,
se procede a integrar los datasets en una única tabla panel.

Cada fila representa una combinación única (`geo`, `TIME_PERIOD`) y cada columna una variable económica.


In [33]:
from functools import reduce

dfs_para_merge = [
    dataframes_sin_duplicados["PSNW"],
    dataframes_sin_duplicados["Labor_Slack"],
    dataframes_sin_duplicados["Inflation_Monthly_Euro_Zone"],
    dataframes_sin_duplicados["GDP_Growth_Rate"],
    dataframes_sin_duplicados["EU_PSNW_over_GDP"],
    dataframes_sin_duplicados["Debt_to_GDP_Ratio"]
]

df_master = reduce(
    lambda left, right: pd.merge(
        left,
        right,
        on=["geo", "TIME_PERIOD"],
        how="outer"
    ),
    dfs_para_merge
)

df_master.shape


(7935, 8)

### 2.5 Verificación final del dataset maestro

Se valida que el dataset maestro contenga una única observación por combinación
(`geo`, `TIME_PERIOD`) y que las variables se encuentren correctamente alineadas.


In [34]:
df_master[["geo", "TIME_PERIOD"]].duplicated().sum()


np.int64(0)

In [35]:
df_master.head()


,geo,TIME_PERIOD,psnw,labor_slack,inflation,gdp_growth,psnw_gdp_ratio,debt_gdp_ratio
0,AL,2013,NaN,NaN,NaN,1.80,NaN,NaN
1,AL,2014,NaN,NaN,NaN,2.35,NaN,NaN
2,AL,2015,NaN,NaN,NaN,2.35,NaN,NaN
3,AL,2016,NaN,NaN,NaN,4.00,NaN,NaN
4,AL,2017,NaN,NaN,NaN,3.35,NaN,NaN


En esta etapa se diagnostica la presencia de valores faltantes resultantes de la integración estructural.

No se aplican aún técnicas de imputación.


En esta etapa se diagnostica la presencia de valores faltantes resultantes de la integración estructural de los distintos datasets.

Los valores faltantes no se consideran errores, sino consecuencia natural de:
- diferencias de cobertura temporal,
- diferencias de cobertura geográfica,
- distinta frecuencia de publicación entre indicadores.

En este punto no se aplican técnicas de imputación.


In [39]:
df_master.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7935 entries, 0 to 7934
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   geo             7935 non-null   object 
 1   TIME_PERIOD     7935 non-null   object 
 2   psnw            930 non-null    float64
 3   labor_slack     2412 non-null   float64
 4   inflation       4113 non-null   float64
 5   gdp_growth      498 non-null    float64
 6   psnw_gdp_ratio  870 non-null    float64
 7   debt_gdp_ratio  1260 non-null   float64
dtypes: float64(6), object(2)
memory usage: 496.1+ KB


In [40]:
df_master.isna().mean().sort_values(ascending=False)


gdp_growth        0.937240
psnw_gdp_ratio    0.890359
psnw              0.882798
debt_gdp_ratio    0.841210
labor_slack       0.696030
inflation         0.481664
geo               0.000000
TIME_PERIOD       0.000000
dtype: float64

## 2.6. Interpretación estructural de los valores faltantes

El patrón de valores faltantes muestra una alta heterogeneidad entre variables:

- Algunas variables presentan cobertura muy limitada en el conjunto total de combinaciones (geo, TIME_PERIOD).
- Otras variables tienen una cobertura intermedia.
- Las claves estructurales (geo, TIME_PERIOD) no presentan valores faltantes.

Este comportamiento es consistente con la integración de fuentes macroeconómicas heterogéneas y confirma que el dataset maestro es estructuralmente correcto.


## 2.7 . Definición del conjunto de variables disponibles para modelado

A partir del dataset maestro integrado, se identifican explícitamente las variables económicas disponibles para el modelado supervisado.

En esta etapa no se decide aún qué variables serán finalmente utilizadas, únicamente se documenta su disponibilidad.


In [41]:
df_master.columns.tolist()


['geo',
 'TIME_PERIOD',
 'psnw',
 'labor_slack',
 'inflation',
 'gdp_growth',
 'psnw_gdp_ratio',
 'debt_gdp_ratio']

Dado el elevado porcentaje de valores faltantes en algunas variables, el tratamiento se realizará en etapas posteriores bajo los siguientes principios:

- La imputación se realizará únicamente sobre el conjunto de entrenamiento.
- No se utilizará información futura para imputar observaciones pasadas.
- Se evaluarán estrategias diferenciadas según el tipo de variable.

Estas decisiones se implementarán en el notebook específico de imputación y preparación final para modelado.


## 2.8 Normalización de nombres de variables para modelado

Con el objetivo de garantizar compatibilidad directa con el notebook de modelado y facilitar la reutilización de código estadístico, se normalizan los nombres de las variables del dataset maestro.

Este ajuste es exclusivamente nominal y no implica transformación de los datos.


In [42]:
# Normalización de nombres de variables para modelado
df_master = df_master.rename(columns={
    "TIME_PERIOD": "time_period",
    "psnw_gdp_ratio": "psnw_gdp",
    "debt_gdp_ratio": "debt_gdp"
})

df_master.columns.tolist()


['geo',
 'time_period',
 'psnw',
 'labor_slack',
 'inflation',
 'gdp_growth',
 'psnw_gdp',
 'debt_gdp']

Se valida que el dataset maestro mantenga su integridad estructural tras el ajuste nominal de variables.


In [43]:
df_master.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7935 entries, 0 to 7934
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   geo          7935 non-null   object 
 1   time_period  7935 non-null   object 
 2   psnw         930 non-null    float64
 3   labor_slack  2412 non-null   float64
 4   inflation    4113 non-null   float64
 5   gdp_growth   498 non-null    float64
 6   psnw_gdp     870 non-null    float64
 7   debt_gdp     1260 non-null   float64
dtypes: float64(6), object(2)
memory usage: 496.1+ KB


In [44]:
df_master.head()


,geo,time_period,psnw,labor_slack,inflation,gdp_growth,psnw_gdp,debt_gdp
0,AL,2013,NaN,NaN,NaN,1.80,NaN,NaN
1,AL,2014,NaN,NaN,NaN,2.35,NaN,NaN
2,AL,2015,NaN,NaN,NaN,2.35,NaN,NaN
3,AL,2016,NaN,NaN,NaN,4.00,NaN,NaN
4,AL,2017,NaN,NaN,NaN,3.35,NaN,NaN


In [46]:
# Exportar dataset maestro a CSV
df_master.to_csv("df_master_ml_ready.csv", index=False)

# 3. Conclusiones del proceso de construcción del dataset maestro

## ¿Qué se hizo?

Durante este notebook se llevó a cabo la construcción completa de un dataset maestro unificado a partir de múltiples fuentes macroeconómicas heterogéneas. El proceso incluyó:

Identificación de columnas estructurales comunes entre datasets.

Reducción de cada dataset a una estructura mínima compatible.

Renombrado nominal de variables para asegurar coherencia semántica.

Diagnóstico y resolución estructural de duplicados por combinación (geo, time_period).

Normalización de la variable temporal time_period como clave estructural homogénea.

Integración progresiva de los datasets en una única tabla panel.

Verificación final de unicidad, integridad estructural y disponibilidad de variables.

El resultado es un dataset maestro con una fila por país y periodo, y una columna por variable económica.

## 3.1 ¿Para qué se hizo?

El dataset maestro construido en este notebook tiene como objetivo servir como base única y reutilizable para las etapas posteriores del proyecto, específicamente:

Feature Engineering predictivo (lags T-1, T-2, T-4).

Definición del target y separación de conjuntos de entrenamiento y validación.

Imputación de valores faltantes bajo criterios estrictos de entrenamiento.

Escalado, selección de variables y modelado supervisado.

Al finalizar este notebook, el proyecto dispone de una estructura panel consistente, trazable y documentada, lista para ser utilizada directamente en el notebook de preparación final y modelado, sin necesidad de reprocesar las fuentes originales.

En esta etapa no se realizan imputaciones, eliminaciones de observaciones ni transformaciones estadísticas, ya que dichas decisiones dependen del esquema de entrenamiento y se implementarán exclusivamente en el notebook de modelado, garantizando así una separación correcta de responsabilidades y evitando contaminación de información futura.